In [3]:
import os
import json
import string
import re  
import csv
import pandas, numpy, string

labels, Text = [], []

path = '/home/raj/UB_Stuff/CSE_635/Final_Data/Thailand/Protest.csv'
for file in os.listdir(path):
     xpath = os.path.join(path,file)
     if xpath.endswith('.json'):
         with open(xpath, "r+") as f:
            data = json.load(f)

         temp = data["text"]
         
         if temp is not None: 
             processed_article = temp.lower()  
             processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )  
             processed_article = re.sub(r'\s+', ' ', processed_article)
             Text.append(processed_article)
             labels.append(data["label"])
                
                
path = '/home/raj/UB_Stuff/CSE_635/Final_Data/Thailand/Violence.csv'
for file in os.listdir(path):
     xpath = os.path.join(path,file)
     if xpath.endswith('.json'):
         with open(xpath, "r+") as f:
            data = json.load(f)

         temp = data["text"]
         
         if temp is not None: 
             processed_article = temp.lower() 
             processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )
             processed_article = re.sub(r'\s+', ' ', processed_article)
             Text.append(processed_article)
             labels.append(data["label"])

path = '/home/raj/UB_Stuff/CSE_635/Phase2/Invalid_Data/Windows/Windows_Invalid_json'
for file in os.listdir(path):
     xpath = os.path.join(path,file)
     if xpath.endswith('.json'):
         with open(xpath, "r+") as f:
            data = json.load(f)

         temp = data["text"] 
         
         
         if temp is not None: 
             processed_article = temp.lower() 
             processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )
             processed_article = re.sub(r'\s+', ' ', processed_article)
             Text.append(processed_article)
             labels.append(data["label"])


# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['Text'] = Text
trainDF['Score'] = labels

text
With 280 rallies and 3,000 roadside meetings planned for over a month, the Aam Aadmi Party AAP on Tuesday said it will launch its campaign for the Lok Sabha elections in Delhi on March 23. AAP s election campaign will begin from March 23 with the first rally scheduled in West Delhi. A total of 280 rallies and 3,000 meetings near polling stations will be held until April 30. AAP s national convener Arvind Kejriwal will hold 70 mega rallies one in each of the city s assembly constituencies, AAP Delhi unit convener Gopal Rai told reporters. With the slogan poorn rajya banao, jhadu ka button dabao vote for AAP and get full statehood , the party said it will distribute Delhi chief minister and AAP chief Arvind Kejriwal s letter explaining the need for statehood to Delhi. It promised Delhi the status within two years if AAP won all seven seats. It also promised a house in 10 years to every family in the city. Kejriwal has also promised the party would introduce 85 reservations for Delhi

##Sampling Data
Sampling dataset to bring each of the classes into same frequency

In [6]:
import pandas as pd
import numpy as np

np.random.seed(0)

def sampling_dataset(df):
    count = 5000
    class_df_sampled = pd.DataFrame(columns = ["Score","Text"])
    temp = []
    for c in df.Score.unique():
        class_indexes = df[df.Score == c].index
        random_indexes = np.random.choice(class_indexes, count, replace=True)
        temp.append(df.loc[random_indexes])
        
    for each_df in temp:
        class_df_sampled = pd.concat([class_df_sampled,each_df],axis=0)
    
    return class_df_sampled

df = trainDF
df = sampling_dataset(df)
df.reset_index(drop=True,inplace=True)
print (df.head())
print (df.shape)

  Score                                               Text
0     1  Jhajjar Women of Kablana village on Friday dis...
1     1  The January story corrects last paragraph to s...
2     1  pune city congress held a protest march agains...
3     1  chennai marina beach in chennai was deserted o...
4     1  the sundarpada market near biju patnaik airpor...
(15000, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


##Train Dov2Vec##
Train Doc2Vec considering each comment as a document. We have taken out words from the comments and then created a LabelSentence object to train Doc2Vec on a size of 300 and variable learning rate sequentially decreasing.

In [9]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re

lmtzr = WordNetLemmatizer()
w = re.compile("\w+",re.I)

def label_sentences(df):
    labeled_sentences = []
    for index, datapoint in df.iterrows():
        tokenized_words = re.findall(w,datapoint["Text"].lower())
        labeled_sentences.append(LabeledSentence(words=tokenized_words, tags=['SENT_%s' %index]))
    return labeled_sentences

def train_doc2vec_model(labeled_sentences):
    model = Doc2Vec(alpha=0.025, min_alpha=0.025)
    model.build_vocab(labeled_sentences)
    for epoch in range(10):
        model.train(labeled_sentences, total_examples=model.corpus_count, epochs=model.epochs)
        model.alpha -= 0.002 
        model.min_alpha = model.alpha
    
    return model

sen = label_sentences(df)
model = train_doc2vec_model(sen)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  from ipykernel import kernelapp as app


## Vectorize Comment ##
Vectorize comments using doc2vec trained model

In [10]:
def vectorize_comments(df,d2v_model):
    y = []
    comments = []
    for i in range(0,df.shape[0]):
        label = 'SENT_%s' %i
        comments.append(d2v_model.docvecs[label])
    df['vectorized_comments'] = comments
    
    return df

df = vectorize_comments(df,model)
print (df.head(2))

  Score                                               Text  \
0     1  Jhajjar Women of Kablana village on Friday dis...   
1     1  The January story corrects last paragraph to s...   

                                 vectorized_comments  
0  [0.32213044, 0.3524667, 1.0661093, -0.3535181,...  
1  [-1.3255713, 2.2842255, -0.8726093, -2.5380015...  


In [26]:
# print (model)
# print type((model.docvecs))
# print (model.docvecs['SENT_0'])
df["vectorized_comments"].T.tolist()
# print (model.docvecs.most_similar('SENT_0'))
# print (df.ix[0]['Text'])
# print ("-----------------------------------------------------")
# print (df.ix[366]['Text'])
# print (model.most_similar(SENT_0))

[array([ 0.32213044,  0.3524667 ,  1.0661093 , -0.3535181 , -1.734332  ,
        -1.3121082 , -0.2690696 ,  0.80978316, -0.4986325 , -0.3561248 ,
         1.3325244 , -0.12563755, -0.408732  ,  0.06620055,  0.09550695,
        -1.0703684 ,  1.0755728 , -0.1787612 , -2.3816974 , -0.5376617 ,
        -0.01221943,  0.82882094,  0.21470706, -1.7850105 ,  1.1169189 ,
         1.2706772 ,  0.5817385 , -0.78832984,  1.2146152 , -0.6579581 ,
         0.44231045,  2.1382513 ,  3.183605  ,  1.0442307 ,  2.5284343 ,
        -1.3671439 , -0.02581223, -1.0721142 ,  1.2128092 ,  0.7947353 ,
         1.7222662 ,  1.9122543 , -0.6762324 ,  1.1178279 ,  1.0046258 ,
         1.1251698 ,  0.95773154,  1.3168272 , -0.03634816,  0.20437157,
        -1.0776447 , -0.7557614 , -0.09435304,  3.4983416 , -0.31128228,
        -0.7180926 , -0.10835484,  0.3697508 ,  0.94761026, -2.0732143 ,
        -0.70248526, -0.28329855,  1.0205282 ,  0.12089678,  1.2218808 ,
        -0.9090449 ,  1.3504171 ,  1.3025602 ,  0.2

In [33]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df["vectorized_comments"].T.tolist(), df["Score"], test_size=0.02, random_state=17)
print(type(X_train))

<class 'list'>


In [37]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train.astype(int))

score = classifier.score(X_test, y_test.astype(int))
print("Accuracy:", score)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.7166666666666667


## Train Random forest classifier on a grid search parameters ##
Training RFC classifier with grid search to tune hyperparameters . Score on test data and highest CV score  printed.

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn import model_selection
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pickle

def train_classifier(X,y):
    n_estimators = [200,400]
    min_samples_split = [2]
    min_samples_leaf = [1]
    bootstrap = [True]

    parameters = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf,
                  'min_samples_split': min_samples_split}

    clf = GridSearchCV(RFC(verbose=1,n_jobs=4), cv=4, param_grid=parameters)
    clf.fit(X, y)
    return clf

X_train, X_test, y_train, y_test = model_selection.train_test_split(df["vectorized_comments"].T.tolist(), df["Score"], test_size=0.02, random_state=17)
#train_test_split(trainDF['text'], trainDF['label'], test_size=0.33, random_state=42)
classifier = train_classifier(X_train,y_train.astype(int))
print (classifier.best_score_, "----------------Best Accuracy score on Cross Validation Sets")
print (classifier.score(X_test,y_test.astype(int)))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  56 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    4.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 144 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    4.1s finished
[Parallel(n_jobs=4)]: Using backend Threadin

0.9942857142857143 ----------------Best Accuracy score on Cross Validation Sets
0.9966666666666667


[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:   16.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.1s finished


In [41]:
f = open("Doc2VecOutput.txt","w")
f.write("Best Accuracy score on Cross Validation Sets %f" %classifier.best_score_,)
f.write("Score on Test Set %f" %classifier.score(X_test,y_test.astype(int)))
f.close()

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.1s finished


In [ ]:
import pickle
# save the model to disk
filename = '/home/raj/UB_Stuff/CSE_635/Phase4/phasr_4_model.sav'
pickle.dump(classifier, open(filename, 'wb'))